In [ ]:
import os
import sys
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.DEBUG)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -q ruamel.yaml
    !pip install -q tensorboard-plugin-profile
    project_path = '/content/drive/MyDrive/Colab Projects/QuantumFlow'
except:
    project_path = os.path.expanduser('~/QuantumFlow')

In [ ]:
os.chdir(project_path)
sys.path.append(project_path)

import numpy as np
import tensorflow as tf
import tree

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import quantumflow

experiment = 'crazynet'
run_name = 'default'

base_dir = os.path.join(project_path, "experiments", experiment)
params = quantumflow.utils.load_yaml(os.path.join(base_dir, 'hyperparams.yaml'))[run_name]
run_dir = os.path.join(base_dir, run_name)

In [ ]:
dataset_train = quantumflow.instantiate(params['dataset_train'], run_dir=run_dir)
dataset_train.build()

dataset_validate = quantumflow.instantiate(params['dataset_validate'], run_dir=run_dir)
dataset_validate.build()

dataset_train.visualize()

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(params['seed'])

model = quantumflow.instantiate(params['model'], run_dir=run_dir, dataset=dataset_train)
display(model.summary())

In [ ]:
steps = 10

def loss_fn(outputs, targets):
    return tf.reduce_mean(tf.square(outputs['kinetic_energy_density'] - targets['kinetic_energy_density']))


def validation_fn():
    outputs_validate = tree.map_structure(lambda out: [], dataset_validate.targets)

    for index in range(dataset_validate.dataset_size):
        features = tree.map_structure(lambda inp: inp[index:index+1], dataset_validate.features)    
        tree.map_structure_up_to(dataset_validate.targets, lambda pred, arr: arr.append(pred.numpy()), model(features), outputs_validate)

    outputs_validate = tree.map_structure_up_to(dataset_validate.targets, lambda arr: np.concatenate(arr), outputs_validate)

    return tree.map_structure(lambda pred, target: np.mean(np.abs(pred - target)), outputs_validate, dataset_validate.targets)


optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

if 'derivative' in dataset_validate.targets:
    del dataset_validate.targets['derivative']
        
losses = []
validation = tree.map_structure(lambda out: [], dataset_validate.targets)

In [ ]:
@tf.function
def step_fn(features, targets):
    with tf.GradientTape() as tape:

        outputs = model(features, training=True) 

        loss_value = loss_fn(outputs, targets)

    grads = tape.gradient(loss_value, model.trainable_weights)

    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value


print(f"{'step':>10} {'loss':>20} {'kinetic_energy':>20} {'kinetic_energy_density':>25}")

for step in range(steps):
    
    index = np.random.randint(100)
    features = tree.map_structure(lambda inp: inp[index:index+1], dataset_train.features)
    targets = tree.map_structure(lambda inp: inp[index:index+1], dataset_train.targets)

    loss_value = step_fn(features, targets)
    
    if step % 1000 == 0:
        mean_abs_err = validation_fn()
        print(f"{step:10d} {loss_value:20f} {mean_abs_err['kinetic_energy']:20f} {mean_abs_err['kinetic_energy_density']:25f}")
        tree.map_structure_up_to(dataset_validate.targets, lambda err, arr: arr.append(err), mean_abs_err, validation)
        
    losses.append(loss_value.numpy())
    
    if len(losses) > 25000:
        optimizer.learning_rate.assign(optimizer.learning_rate*0.9999)
        
# kinetic_energy 0.00004
# kinetic_energy_density 0.006

In [ ]:
plt.figure(figsize=(20, 3))
plt.plot(losses)
plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(20, 3))
plt.plot(validation['kinetic_energy'])
plt.yscale('log')
plt.show()

plt.figure(figsize=(20, 3))
plt.plot(validation['kinetic_energy_density'])
plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(20, 3))
for index in range(5):
    features = tree.map_structure(lambda inp: inp[index:index+1], dataset_validate.features)
    targets = tree.map_structure(lambda inp: inp[index:index+1], dataset_validate.targets)
    
    outputs = model(features)
    plt.plot(dataset_train.x, outputs['kinetic_energy_density'].numpy().transpose())
    plt.plot(dataset_train.x, targets['kinetic_energy_density'].transpose(), 'k')
plt.show()

In [ ]:
plt.figure(figsize=(20, 3))
for index in range(5):
    features = tree.map_structure(lambda inp: inp[index:index+1], dataset_validate.features)
    targets = tree.map_structure(lambda inp: inp[index:index+1], dataset_validate.targets)
    
    outputs = model(features)
    plt.plot(dataset_train.x, outputs['kinetic_energy_density'].numpy().transpose()-targets['kinetic_energy_density'].transpose(), 'k')
plt.show()

In [ ]:
validation_fn()